In [1]:
import sys

sys.path.append(r'C:\users\17049\appdata\local\programs\python\python311\lib\site-packages')

import pandas as pd
import pandas_ta as ta
import fredapi as fa
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Source: https://fred.stlouisfed.org/
fred = fa.Fred(api_key =os.environ.get('FRED_API_KEY'))

#Get UST10 Yield
ten = fred.get_series('DGS10')
ten.name = 'UST 10 Year Yield'
df = pd.DataFrame(ten)

#Get ave 30 yr mortgage data
dfm = fred.get_series('MORTGAGE30US')
dfm.name = 'Ave US 30 Year Mortgage Rate'
dfm = pd.DataFrame(dfm)

m = pd.merge(df, dfm, left_index=True, right_index=True).dropna()

m['Spread'] = m['Ave US 30 Year Mortgage Rate'] - m['UST 10 Year Yield']

m.tail(10)

,UST 10 Year Yield,Ave US 30 Year Mortgage Rate,Spread
2023-08-03,4.20,6.90,2.70
2023-08-10,4.09,6.96,2.87
2023-08-17,4.30,7.09,2.79
2023-08-24,4.23,7.23,3.00
2023-08-31,4.09,7.18,3.09
2023-09-07,4.27,7.12,2.85
2023-09-14,4.29,7.18,2.89
2023-09-21,4.49,7.19,2.70
2023-09-28,4.59,7.31,2.72
2023-10-05,4.72,7.49,2.77


In [4]:
#Construct chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.update_traces(marker_color='lightblue', opacity=0.5)
fig.add_trace(go.Scatter(x=m.index, y=m['Spread'], mode='lines', name='Mortgage - 10 Year', marker_color='navy'))

fig.update_layout(barmode='stack', title_text='30 Yr Mortage / UST10 Basis - w/Rangeslider', showlegend=False,
                  hoverlabel=dict(bgcolor='white'), template='plotly_white', xaxis=dict(showgrid=False), 
                  yaxis=dict(title='Mortgage Treasury Spread (bps)',tickformat=',.2f', showgrid=False),
                  xaxis_rangeslider_visible=True)

fig.add_annotation(text='Data: https://fred.stlouisfed.org/',
                  xref='paper', yref='paper',
                  x=0.92, y=-0.15, showarrow=False)

fig.show()

In [5]:
fig.write_html(r'C:\Users\17049\Documents\Charts\index.html', full_html=False, include_plotlyjs='cdn')